<a href="https://colab.research.google.com/github/Teganmosi/Tweets-classification/blob/main/nlp_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7e6007f6-021e-4665-a77e-d316dbc85bb3)


In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-22 21:09:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-22 21:09:48 (102 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-23 09:43:27--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.96.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.31MB/s    in 0.4s    

2023-06-23 09:43:27 (1.31 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
print(f'Total training samples: {len(train_df)}')
print(f'Total Test samples: {len(test_df)}')

Total training samples: 7613
Total Test samples: 3263


In [ ]:
# to visualize some examples in the training set
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target:{target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---\n')


Target:1 (real disaster)
Text:
EUROCRISIS Guardian Greece's tax revenues collapse as debt crisis continues: As talks continue over proposed âÂ... http://t.co/bBm9sR1wOw

---

Target:0 (not real disaster)
Text:
I'm about to cook your Smokey the Bear saving forest fires face ass  https://t.co/WtGGqS5gEh

---

Target:1 (real disaster)
Text:
Û÷Hijacker copsÛª back in the dock - http://t.co/9I5cczD5S0 http://t.co/WEaTrRihE1

---

Target:0 (not real disaster)
Text:
#poster #ergo Rotator #cuff injury recovery kit: http://t.co/zj3ODGQHyp Super High Converting Rotator Cuff Inj http://t.co/VZhTiBe4jh

---

Target:0 (not real disaster)
Text:
If you dotish to blight your car go right ahead. Once it's not mine.

---



In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [ ]:
from numpy.random import standard_t
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None)

In [ ]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectoriser = TextVectorization(max_tokens = max_vocab_length,
                  standardize = 'lower_and_strip_punctuation',

                  output_mode = 'int',
                  output_sequence_length = max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectoriser.adapt(train_sentences)

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectoriser([random_sentence])

Original text:
TWISTER DANCE Game  Dance Console Instructions Cable 5 Pre Loaded Songs http://t.co/BcsXyEc4ji http://t.co/DfXI76kvX0      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 358, 1307,  397, 1307, 5932,    1, 1317,  180, 4878, 5206, 3398,
           1,    1,    0,    0]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectoriser.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
from tensorflow.keras import layers
tf.random.set_seed(42)

embedding = layers.Embedding(input_dim = max_vocab_length,
                output_dim = 128,
                embeddings_initializer = 'uniform',
                input_length = max_length,
                name = 'embedding_1')

embedding

In [ ]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectoriser([random_sentence]))
sample_embed

Original text:
Just demolished a Snowball ??      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00712768,  0.04700074,  0.01009767, ..., -0.01365947,
         -0.00357716,  0.01146861],
        [-0.00804063, -0.00431185,  0.04411607, ...,  0.03241532,
          0.04126364, -0.03372001],
        [ 0.03954938, -0.01215825,  0.04626428, ...,  0.03386058,
         -0.02681305, -0.03199029],
        ...,
        [-0.01404071,  0.00373019, -0.02270451, ...,  0.02614769,
          0.01415061,  0.01571028],
        [-0.01404071,  0.00373019, -0.02270451, ...,  0.02614769,
          0.01415061,  0.01571028],
        [-0.01404071,  0.00373019, -0.02270451, ...,  0.02614769,
          0.01415061,  0.01571028]]], dtype=float32)>

In [ ]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.00712768,  0.04700074,  0.01009767, -0.03608022,  0.0398815 ,
        0.01895941,  0.01357253,  0.04454801,  0.02618309, -0.01194942,
       -0.04157228, -0.04639225,  0.034533  ,  0.00699208, -0.03997465,
       -0.00309227,  0.02200147,  0.03932727,  0.01139291,  0.0284356 ,
        0.03327124, -0.04780943,  0.00857624,  0.04423478,  0.01114585,
        0.01886788,  0.04550654,  0.02557534,  0.00563102, -0.04356098,
       -0.00117443, -0.00386841,  0.04769815,  0.00167141, -0.04898871,
        0.0426701 ,  0.01008825, -0.01137972, -0.00265916, -0.02010067,
        0.03316658, -0.01461142, -0.04890027, -0.01955211, -0.00656521,
       -0.03911514,  0.04929027,  0.01816121,  0.0330965 , -0.00465528,
        0.02378175, -0.04700785, -0.02384781,  0.04553946,  0.04513692,
       -0.00471222,  0.0426667 , -0.01176406, -0.03789731,  0.01873452,
        0.02729804, -0.01705562, -0.0326758 , -0.01426265,  0.03991078,
       -0.025623

In [ ]:
from math import pi
#Creating the baseline model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('cif', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('cif', MultinomialNB())])

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Our baseline model achieves an accuracy of :{baseline_score*100:.2f}%')

Our baseline model achieves an accuracy of :79.27%


In [ ]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
from tensorflow.keras import layers
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [ ]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectoriser(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 =  tf.keras.Model(inputs, outputs, name = 'model_1_dense')

In [ ]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = 'simple_dense')])

Saving TensorBoard log files to: model_logs/simple_dense/20230622-210959
Epoch 1/5
215/215 [==============================] - 18s 61ms/step - loss: 0.6105 - accuracy: 0.6900 - val_loss: 0.5361 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4415 - accuracy: 0.8175 - val_loss: 0.4689 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3466 - accuracy: 0.8608 - val_loss: 0.4588 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2851 - accuracy: 0.8926 - val_loss: 0.4638 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2382 - accuracy: 0.9123 - val_loss: 0.4762 - val_accuracy: 0.7861


In [ ]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4762 - accuracy: 0.7861


[0.47623270750045776, 0.7860892415046692]

In [ ]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.00464517, -0.01793496, -0.03343829, ..., -0.01767584,
         -0.04877281,  0.00059422],
        [-0.01756839, -0.04217419, -0.00885879, ...,  0.03378577,
          0.00239203, -0.02629844],
        [ 0.02024213, -0.04241568, -0.04730637, ..., -0.0317949 ,
          0.01354243,  0.01069985],
        ...,
        [ 0.03318632, -0.041209  ,  0.01036185, ...,  0.0248119 ,
          0.03980971,  0.04046804],
        [ 0.06379078, -0.00875837, -0.05405978, ..., -0.04332094,
         -0.06310004, -0.03138165],
        [ 0.0165656 , -0.05279806, -0.04684222, ..., -0.05626651,
         -0.02686615, -0.03197332]], dtype=float32)>]

In [ ]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:20]

24/24 [==============================] - 0s 2ms/step


array([[0.39648885],
       [0.74431324],
       [0.997649  ],
       [0.10790917],
       [0.10353084],
       [0.9361603 ],
       [0.91006494],
       [0.99267465],
       [0.969218  ],
       [0.26720315],
       [0.12279497],
       [0.7113796 ],
       [0.05517568],
       [0.22112456],
       [0.00503994],
       [0.16456874],
       [0.02461431],
       [0.10277135],
       [0.23746568],
       [0.5590359 ]], dtype=float32)

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [ ]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred = model_1_preds)

model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [ ]:
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [ ]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [ ]:
words_in_vocab = text_vectoriser.get_vocabulary()
len(words_in_vocab), words_in_vocab[:20]

(10000,
 ['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this'])

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
# Get the weight matrix of embedding layer
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
#Using RNN
tf.random.set_seed(42)
model_2_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_2')

#creating LSTM model
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectoriser(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)

(None, 15, 128)
(None, 64)


In [ ]:
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [ ]:
model_2.fit(train_sentences,
          train_labels,
          epochs = 5,
          validation_data = (val_sentences, val_labels),
          callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                   'LSTM')])

Saving TensorBoard log files to: model_logs/LSTM/20230622-211042
Epoch 1/5
215/215 [==============================] - 16s 57ms/step - loss: 0.5104 - accuracy: 0.7435 - val_loss: 0.4548 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3170 - accuracy: 0.8705 - val_loss: 0.5091 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2190 - accuracy: 0.9174 - val_loss: 0.5886 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1557 - accuracy: 0.9450 - val_loss: 0.6594 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1101 - accuracy: 0.9591 - val_loss: 0.7662 - val_accuracy: 0.7598


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


((762, 1),
 array([[0.13997449],
        [0.7981125 ],
        [0.9984909 ],
        [0.08421998],
        [0.00505279],
        [0.99583924],
        [0.8466705 ],
        [0.9990489 ],
        [0.9984289 ],
        [0.40395084]], dtype=float32))

In [ ]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.98425196850394,
 'precision': 0.7602880613964507,
 'recall': 0.7598425196850394,
 'f1': 0.7582369337868397}

In [ ]:
# Compare model 2 to baseline
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 75.98, Difference: -3.28
Baseline precision: 0.81, New precision: 0.76, Difference: -0.05
Baseline recall: 0.79, New recall: 0.76, Difference: -0.03
Baseline f1: 0.79, New f1: 0.76, Difference: -0.03


In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
model_3.compile(loss = 'binary_crossentropy')